In [28]:
import json
import time

from src.experiments.aux_code import *
from datasets.get_datasets import *
import time
from src.genetic import *
from src.predicate import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Testing groot to many predicates

In [22]:
predicates = {'source': ['workedunder', 'movie', 'genre'],
              'target': ['advisedby', 'publication', 'inphase']}

In [29]:
bk = json.loads(open('src/experiments/kb.txt').readline())


In [30]:
final_results = {}
source_pred = predicates['source']
target_pred = predicates['target']

for pred in zip(source_pred, target_pred):
    final_results[f'{pred[0]}->{pred[1]}'] = ''
    final_results[f'inf_res:{pred[0]}->{pred[1]}'] = ''
    final_results[f'groot_test:{pred[0]}->{pred[1]}'] = ''
    final_results[f'tree_test:{pred[0]}->{pred[1]}'] = ''

In [35]:
kb_source = bk['imdb']
kb_target = bk['uwcse']

In [36]:
pred_target = create_pred_target(bk_target)

In [38]:
for source, target in zip(source_pred, target_pred):
    print(f'Iniciando transferência {source}->{target}')

    imdb_dataset = datasets.load('imdb', kb_source, target=source, seed=441773) #facts, pos, neg
    uwcse_dataset = datasets.load('uwcse', kb_target, target=target, seed=441773) #facts, pos, neg

    train_facts_source, train_pos_source, train_neg_source = get_train_division(imdb_dataset)
    background_knowledge_src = boostsrl.modes(kb_source, [source], useStdLogicVariables=False, 
                                          maxTreeDepth=3, nodeSize=2, numOfClauses=8)
    model_src = boostsrl.train(background_knowledge_src, train_pos_source, train_neg_source, train_facts_source, trees=10)
    
    structured_src, new_src_struct = create_structured_trees(model_src)
    
    
    train_facts, train_pos, train_neg, test_facts, test_pos, test_neg = get_train_test(uwcse_dataset)
    
    start = time.time()
    final_results[f'{source}->{target}'] = genetic(new_src_struct, target, source, train_pos, 
                                             train_neg, train_facts, kb_source, kb_target, pred_target,
                                              NUM_GEN=14, pop_size=30)
    print("FIM DO GROOT: ", time.time()-start)
    
    tr_train_pos = []
    tr_train_neg = []
    tr_train_facts = []

    for i in range(0, len(train_pos)):
        tr_train_pos.extend(train_pos[i])
        tr_train_neg.extend(train_neg[i])
        tr_train_facts.extend(train_facts[i])
        
    train = [tr_train_pos, tr_train_neg, tr_train_facts]
    test = [test_pos, test_neg, test_facts]
    
    individual = get_best_individual(final_results[f'{source}->{target}'][0].population)
    refine, transfer = get_refine_transfer(individual, source, target, 'imdb', 'uwcse')
    
    final_results[f'groot_test:{source}->{target}'], model = test_refine_transfer(kb_target, target, refine, transfer, train, test)
    
    final_results[f'inf_res:{source}->{target}'] = model.inference_results(target)
    
    final_results[f'tree_test:{source}->{target}'] = test_tree_b(source, target, kb_source, kb_target, structured_src, train, test)
    
    save_results(final_results, source, target, 'imdb', 'uwcse')
    
    print(f"Finalizando transferência {source}->{target} com tempo {time.time()-start} s")

Iniciando transferência workedunder->advisedby
GENERATION:  0
MELHOR RESULTADO:  0.28873125
GENERATION:  1
MELHOR RESULTADO:  0.2707865
GENERATION:  2
MELHOR RESULTADO:  0.26723425
GENERATION:  3
MELHOR RESULTADO:  0.26723425
GENERATION:  4
MELHOR RESULTADO:  0.25451949999999995
GENERATION:  5
MELHOR RESULTADO:  0.25451949999999995
GENERATION:  6
MELHOR RESULTADO:  0.25451949999999995
GENERATION:  7
MELHOR RESULTADO:  0.25451949999999995
GENERATION:  8
MELHOR RESULTADO:  0.249064
GENERATION:  9
MELHOR RESULTADO:  0.249064
GENERATION:  10
MELHOR RESULTADO:  0.249064
GENERATION:  11
MELHOR RESULTADO:  0.249064
GENERATION:  12
MELHOR RESULTADO:  0.249064
GENERATION:  13
MELHOR RESULTADO:  0.249064
FIM DO GROOT:  5244.458270072937
******************************************
Performing Parameter Learning
******************************************
Refine
0;;workedunder(A, B) :- actor(A), director(B).;true;false
0;true;movie(C, A), movie(C, B).;false;false
1;;workedunder(A, B) :- director(B), 

GENERATION:  0
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  1
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  2
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  3
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  4
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  5
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  6
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  7
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  8
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  9
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  10
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  11
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  12
MELHOR RESULTADO:  0.6725415000000001
GENERATION:  13
MELHOR RESULTADO:  0.6725415000000001
FIM DO GROOT:  5337.669332027435
******************************************
Performing Parameter Learning
******************************************
Refine
0;;movie(A, B) :- actor(B).;true;false
0;true;female(B).;false;false
1;;movie(A, B) :- actor(B).;t

GENERATION:  0
MELHOR RESULTADO:  0.695026
GENERATION:  1
MELHOR RESULTADO:  0.695026
GENERATION:  2
MELHOR RESULTADO:  0.695026
GENERATION:  3
MELHOR RESULTADO:  0.695026
GENERATION:  4
MELHOR RESULTADO:  0.695026
GENERATION:  5
MELHOR RESULTADO:  0.695026
GENERATION:  6
MELHOR RESULTADO:  0.695026
GENERATION:  7
MELHOR RESULTADO:  0.695026
GENERATION:  8
MELHOR RESULTADO:  0.695026
GENERATION:  9
MELHOR RESULTADO:  0.695026
GENERATION:  10
MELHOR RESULTADO:  0.695026
GENERATION:  11
MELHOR RESULTADO:  0.695026
GENERATION:  12
MELHOR RESULTADO:  0.695026
GENERATION:  13
MELHOR RESULTADO:  0.695026
FIM DO GROOT:  4257.78998708725
******************************************
Performing Parameter Learning
******************************************
Refine
0;;genre(A, B) :- actor(A).;false;false
1;;genre(A, B) :- actor(A).;false;false
2;;genre(A, B) :- actor(A).;false;false
3;;genre(A, B) :- actor(A).;false;false
4;;genre(A, B) :- actor(A).;false;false
5;;genre(A, B) :- actor(A).;false;false